# 번역기를 만들어보자.

현재까지 가장 성공적인 인공지닝 어플리케이션 중 하나는 신경망 기계번역이다.
구글 번역기나 파파고등을 쓰고 있는데, 완벽하지는 않아도 매우 높은 정확도로 빠르게 번역해주고 있다.

최근에 기술은 더 좋아졌지만, seq2seq라는 모델이 처음 도입되었을 때, 큰 도약이 이루어졌다. 2014년에 등장하여 오래된 기술 같지만, 그만큼 모델링에 근간이 되기에 실습을 통해서 높은 수준의 번역기를 만들어보자.

## 기계 번역의 역사
40년대 기계번역 연구가 시작되어서, 80년대까지 규칙 기반의 기계번역이 주를 이뤘다. 문법 공부하듯이 규칙을 찾아내야한다. 80년대 후반 IBM에서 통계적 기계번역이라는 모델을 연구했다. 2010년 중반까지 주류를 이뤘다.
인공신경망으로 번역을 제안한 연구는 많았지만, 적은 신경망의 크기와 낮은 단계의 알고리즘과 하드웨어 사양으로 주류 기술이 되지 못했다.

구글이 2016년 9월 구글 번역기에 신경망 기계번역을 도입하면서 획기적인 성능개선을 이뤘다. 이때 사용된 인공 신경망이 seq2seq입니다.

# seq2seq의 시퀀스
![LSTM 인코더와 디코더](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)
<center>*[LSTM 인코더 디코더, 출처 : http://karpathy.github.io/2015/05/21/rnn-effectiveness/]*</center>

sequence to sequence를 의미하는 seq2seq는 한 범주의 시퀀스 자료를 입력받아 다른 범주의 시퀀스 자료로 출력한다는 의미다. 대표적으로 문장 번역에 해당한다. 그럼 시퀀스를 처리하는 RNN에 대해서 알아보자.

## 시퀀스를 처리하는 RNN
![RNN의 종류](https://d3s0tskafalll9.cloudfront.net/media/images/E-15-3.rnn_effectiveness.max-800x600.jpg)
<center>*[RNN의 종류, 출처 : http://karpathy.github.io/2015/05/21/rnn-effectiveness/]*</center>

- 종류
    1. one to one : 순환하지 않는 피드 포워드 신경망 (RNN이 아니다.)
    2. one to many : 이미지를 받아서 이미지의 제목, 캡션을 추출하는 등의 RNN 작업
    3. many to one : 시퀀스를 입력받아서 하나의 결과를 내는 RNN으로 텍스트 분류에 많이 쓰인다.
    4. many to many (delayed) : 시퀀스를 입력받는 동안은 출력을 내지 않다가 어느 기준점부터 출력한다. 변역기에서 주로 활용한다.
    5. many to many : 시퀀스를 입력받는대로 바로 출력한다.
    
## seq2seq의 인코더-디코더 구조
앞서 UNet과 DeepLab V3+에서 인코더-디코더 구조를 통해서 정밀한 예측이 가능하다고 했다.
![seq2seq의 인코더-디코더 구조](https://d3s0tskafalll9.cloudfront.net/media/images/E-15-4.seq2seq.max-800x600.jpg)
<center>*[seq2seq의 인코더-디코더 구조, 출처 : https://notebooks.azure.com/anon-ssuccw/projects/2018-data-access/html/nlp_course/week04_seq2seq/practice.ipynb]*</center>

번역에서 보통, 입력 문장과 출력 문장의 길이가 다르다. 타겟 문장의 예측을 시작하기 위해서는 입력 문장의 전체가 필요하다. 이러한 요구사항 때문에 LSTM은 더 발전된 설정이 seq2seq이다.
seq2seq는 두 개의 RNN(인코더 RNN, 디코더 RNN) 아키텍처를 연결한 구조다
학습된 LSTM 은닉층과 셀스테이트(컨디션)을 디코더에서 사용한다.


## Feature를 기준으로 줄이고 늘리는 인코더-디코더
![feature encoder and feature decoder](https://d3s0tskafalll9.cloudfront.net/media/images/E-15-5.encdec.max-800x600.png)
<center>*[feature encoder and feature decoder, 출처 : http://karpathy.github.io/2015/05/21/rnn-effectiveness/]*</center>

위의 그림처럼 인코더는 특성을 추출해내고, x를 해석하기 위한 저차원의 feature vactor z 를 만든다.
한편, 디코더는 낮은 차원의 feature z로부터 정보를 입력값을 복원한다.


seq2seq2의 feature vector는 인코더 RNN이 입력문장을 해석해서 만들어낸 가중치 백터(은닉층)이다. 문장 x를 z라고 가중치 백터로 인코딩하고 z를 다시 번역하고자 하는 언어로 재생성한다.

디코더는 인코더의 마지막 은닉층을 전달받아 초기 은닉층으로 입력하여 문장을 생성한다. 여기서 출력문장의 시작과 종료를 특수문자로 알려주는데 _GO(시작), EOS(끝)을 표시한다. SOS(start of sequence)와 EOS(end of sequence)로 나타내기도 한다.

## Conditional Language model
문장 생성기 모델과 같이 언어모델(lenguage model)을 구현할 것이다.
언어모델은 단어들이 시퀀스 자료형으로 주어질 때, n-1번째 단어가 왔을 때, n번째 단어가 올 확률을 p로 해서 예측하는 RNN모델들이다.
이런 지난 문장생성기는 첫 단어가 주어지면, 언어 모델은 확률에 기반해서 문장을 만들어 냈다. 어떤 말을 만들고 싶은지 전체 문장을 입력받고, 다른 언어로 출력해야 한다. 기존의 단어들과 theta의 파라미터만 있다면, C라는 원하는 문장을 나타내는 파라미터가 추가된다. 이러한 언어모델을 conditional language model이라고 한다.

영어 문장 x를 인코더 RNN에 넣어서 특성 백터 c를 만들고, 디코더 RNN으로 다시 다른 언어 문장 y로 바꿔주는 모델을 다루는 것이 seq2seq 모델이다.

## 교사 강요 (teacher forcing)

![teaching forcing](https://d3s0tskafalll9.cloudfront.net/media/original_images/E-15-6.teacher_forcing.png)
<center>*[difference before and after Teacher Forcing, 출처 : https://towardsdatascience.com/what-is-teacher-forcing-3da6217fed1c]*</center>

seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다르다.
테스트 과정에서는 이전 은닉층의 값을 다음 레이어에서 사용한다. 하지만 훈련 과정에서는 time step이 아직 정확하지 않기 때문에 이렇게 반영시키면 훈련시간이 늘어진다.
그래서 훈련과정에서 실제 정답 시퀀스를 알고 있기 때문에, 이전의 time step 값을 쓰는게 아니라 시퀀스 time step의 실제값을 사용하는데, 이를 교사 강요라고 한다. 이 기법은 seq2seq 뿐만 아니라 sequence 데이터 생성 모델에서 일반적으로 사용되는 기법이다.

## 단어 수준과 문자 수준 방식의 seq2seq
RNN의 은닉층마다 입출력 단위가 단어이냐 문자이냐에 따라 단어 수준과 문자 수준의 seq2seq인지 구분한다.
모델을 구현하는데 문자 수분이 더 쉬운데, 그 이유는 단어는 입력층과 출력층의 크기가 매우 커지기 때문이다. 하지만 문자수준으로 하면 알파벳이면 26개 밖에 되지 않고, 기호들을 포함해도 100여개 남짓이기 떄문이다.

그렇다면 단어가 유리한가 하면 항상 그런것도 아니다. 각자 장단점이 있기 때문이다. 단어 수준에서 번역의 단점은 같은 의미인데 다른 형태들이 무수히 많다. 영어로도 eat, eats, eatten, ate 등처럼 시제를 다루는 경우나 한국어의 간다, 갔다, 가고, 가지, 가서, 가면, 갈까 등으로 약간씩 다른 의미들을 어떻게 구분하여야 하는지 어려움이 있고, 띄어쓰기를 어떻게 처리해야하나 하는 문제들이 남았다.

문자 수준으로 번역하면 이런 문제가 해결되지만 그말인 즉슨 내재된 정보가 소실된다는 것을 의미한다. 즉, 기계가 글자가 이루는 패턴까지 학습하고 의미를 파악해내야 한다는 것이다. 막대한 학습량을 위한 충분한 데이터가 필요하다. 그렇지 않으면 단어 수준의 번역보다 품질이 떨어진다.

최신 자연어처리의 흐름은 단어 수준이나 문자 수준의 번역이 아닌 그 사이의 subword 기반의 번역이 주를 이루고 있습니다.

# 문자 수준의 번역기 만들기

## 도구 라이브러리 임포팅

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

## 데이터 불러오기

번역기 모델 학습에 사용할 데이터는 [Tab-delimited Bilingual Sentence Pairs](https://www.manythings.org/anki/)에서 'fra-eng.zip'을 다운 받아서 사용한다.

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
152662,Tom is obsessed with power and money.,Tom est obsédé par le pouvoir et l'argent.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
65408,I've brought you a gift.,Je t'ai apporté un cadeau.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
89075,It's a very complex system.,C'est un système très complexe.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
55644,Have you been to Kyoto?,Êtes-vous déjà allé à Kyoto ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
169635,The town is two miles away from the coast.,Cette ville est à deux milles de la côte.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


## 데이터 전처리

### 불필요한 데이터 삭제
세 번째 열인 cc는 번역기 모델학습에 불필요하니 삭제한다.

In [3]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
40268,You must be in love.,Vous devez être amoureuse.
42255,I couldn't care less.,Je m'en soucie comme d'une guigne.
29019,He asked for money.,Il demanda de l'argent.
12616,We're punctual.,Nous sommes ponctuels.
6540,You're funny.,Vous êtes marrante.


### seq2seq 디코더가 활용하는 토큰 넣어주기
디코더 입력과 예측에는 시작 토큰 <sos>, 종료 토큰 <eos>를 넣어줘야 한다.
이번 실습에서는 시작 토큰을 '/t'로 종료 토큰을 '/n'으로 구분하기로 하자.

In [4]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
2903,Don't leave!,\t Ne pars pas ! \n
39862,Where are your kids?,\t Où sont tes enfants ? \n
48842,He rested for a while.,\t Il s'est reposé un moment. \n
33314,What are you doing?,\t Qu'es-tu en train de faire ? \n
13390,Do we have rice?,\t Avons-nous du riz ? \n


### 단어사전 만들기
각 단어에 할당된 고유한 정수로 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩을 한다.
번역하고자 하는 영어와 출력해줘야하는 프랑스어를 따로 구분해서 만들어주자.

#### 영어 단어사전

In [5]:
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 3, 8], [19, 3, 8], [19, 3, 8]]

#### 프랑스어 단어사전

In [6]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[10, 1, 19, 5, 1, 31, 1, 11],
 [10, 1, 15, 5, 12, 16, 29, 2, 14, 1, 11],
 [10, 1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1, 11]]

#### 단어 사전 임배딩 패딩 처리하기
최대길이를 <pad>를 이용하여 통일해주어야 모델에 학습시켜야 한다.
그럼 먼저 최대길이와 단어사전의 통계들을 둘러보자.

In [7]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73


In [8]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 76


In [9]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 76


## 인코더용 디코더용 데이터셋 만들기
인코더로만 입력되는 영어시퀀스와 달리, 디코더에서 활용되는 프랑스어 시퀀스는 디코더의 출력값과 비교해야할 라벨 데이터셋 하나와 'Teacher forcing'을 위해 디코더에 입력되는 데이터다. Teacher forcing을 위해 디코더로 입력하는 시퀀스는 <eos>토큰이 필요없고, 디코더 출력값과 비교할 라벨 시퀀스는 <sos>가 필요없다.
영어로 'I am a person'이라는 문장을 프랑스어 'Je suis une personne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측한다.

In [10]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

영어 단어사전을 인코더 입력 데이터셋으로 만들고, 프랑스 단어사전은 종료 토큰이 제거된 디코더 입력 데이터셋, 시작 토큰이 제거된 디코더 교사 강요 데이터셋 총 3가지 데이터셋을 만든다.

In [11]:
print(decoder_input[:3])
print(decoder_target[:3])

[[10, 1, 19, 5, 1, 31, 1], [10, 1, 15, 5, 12, 16, 29, 2, 14, 1], [10, 1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1]]
[[1, 19, 5, 1, 31, 1, 11], [1, 15, 5, 12, 16, 29, 2, 14, 1, 11], [1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1, 11]]


### 패딩을 만들어주자

최대 길이를 파라미터에 가져와서, 'pre'로 패딩을 만들어주자.

In [12]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


패딩처리된 인코더 입력 데이터를 출력해보자.

In [13]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [14]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22, 53)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


In [15]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 22, 53)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


## 모델 훈련하기
이번 실습은 케라스 창시자 프랑수아 숄레의 [케라스의 seq2seq 구현 가이드 게시물](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)을 참고하였습니다.

In [16]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print('⏳')

⏳


인코더와 디코더 모델을 설계해야한다.

인코더는 LSTM셀을  설계하여 문장을 입력받으면 LSTM셀이 마지막 time step의 은닉층의 셀 스테이트(가중치)를 전달받아서 저장합니다. 앞서 인코더의 마지막 은닉층을 디코더의 첫번째 은닉층을 디코더의 첫번째 은닉층으로 사용하며, 이전 셀 스테이트(가중치)까지 가져온다.

In [17]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

LSTM의 입력 텐서를 정의한다.
인코딩 정보와, eng_vocab_size(최대 단어수를 갖는다.)

은닉층의 파라미터의 사이즈를 256으로 정의한다. LSTM의 수용력(capacity)를 의미하며
return_state= true로 해서 은닉층(히든 스테이트)와 셀 스테이트를 리턴 받도록 한다.

입력 텐서를 모델에 넣어서 인코더 아웃풋과 은닉층, 셀 스테이트를 전달한다.

인코더 스테이트를 만들어 은닉층과 셀스테이트를 할당해줬다.

디코더를 설계해보자.

In [18]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

은닉층과 몇 가지 사항을 제외하고 거의 똑같다.

디코더 LSTM을 보면 이니셜 스테이트가 추가되었다. LSTM 모델의 초기 셀 스테이트를 정의해줄 수 있는 스테이트 이다. 여기서 이전에 저장한 인코더의 마지막 시간 단계의 은닉층과 셀 스테이트를 사용해준다.

매 시간 단계마다 다중 클래스 분류문제 이므로 프랑스어 단어사전으로부터 한 가지 문자만 선택하도록 한다.
FC 레이어의 파라미터로 단어사전의 크기를 기재하고, 다중 클래스 분류 활성화 함수로 소프트맥스 함수를 사용한다.

In [19]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

마지막으로 인코더와 디코더를 연결해서 하나의 모델을 만들어 준다.

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 53)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 317440      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [21]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
368/368 [==============================] - 12s 21ms/step - loss: 0.8835 - val_loss: 0.7662
Epoch 2/50
368/368 [==============================] - 7s 18ms/step - loss: 0.5493 - val_loss: 0.6345
Epoch 3/50
368/368 [==============================] - 7s 18ms/step - loss: 0.4546 - val_loss: 0.5499
Epoch 4/50
368/368 [==============================] - 7s 18ms/step - loss: 0.3975 - val_loss: 0.4950
Epoch 5/50
368/368 [==============================] - 7s 18ms/step - loss: 0.3606 - val_loss: 0.4588
Epoch 6/50
368/368 [==============================] - 7s 19ms/step - loss: 0.3339 - val_loss: 0.4343
Epoch 7/50
368/368 [==============================] - 7s 19ms/step - loss: 0.3134 - val_loss: 0.4208
Epoch 8/50
368/368 [==============================] - 7s 19ms/step - loss: 0.2969 - val_loss: 0.4075
Epoch 9/50
368/368 [==============================] - 7s 18ms/step - loss: 0.2836 - val_loss: 0.3934
Epoch 10/50
368/368 [==============================] - 7s 18ms/step - loss: 0.2720 - val_l

## 모델 테스트하기

seq2seq는 훈련할 때와 테스트 단계의 동작이 다르다.
테스트 단계의 디코더 모델을 다시 설계해줘야 한다.
문자 생성기 모델을 만들어 보신 분이라면 알 수 있지만, 훈련시에는 학습해야 할 타겟 문장을 디코더 모델을 입력, 출력 시퀀스로 넣어주고, 디코더 모델이 타겟 문장을 한꺼번에 출력하게 할 수 있다. 그러나 테스트 단계에서 그럴 수가 없다. 하나의 문장을 만들어 내기 위해 루플를 돌면서 단어를 하나씩 차례로 예측하고, 예측한 단어가 다시 다음 단어를 예측할 때 사용되는 입력으로 재 사용 되는 과정이 진행되기 때문이다.

테스트 단계에서의 디코더의 동작 순서는 아래와 같습니다.

1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.
2. 토큰인 '\t'를 디코더에 입력한다.
3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 한다.
4. 3을 반복하다가 토큰인 '\n'가 예측되면 이를 중단한다

우선 앞서 정의한 'encoder_inputs'과 'encoder_states'를 사용해서 인코더를 정의하자.

In [22]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 53)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 317440    
Total params: 317,440
Trainable params: 317,440
Non-trainable params: 0
_________________________________________________________________


이제 디코더를 설계하자

In [23]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

디코더의 출력층을 재설계하자

In [24]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_3[0][0]              

In [25]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [26]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [27]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:  va ! 
-----------------------------------
입력 문장: Hello!
정답 문장:  Bonjour ! 
번역기가 번역한 문장:  souvez ! 
-----------------------------------
입력 문장: Got it?
정답 문장:  T'as capté ? 
번역기가 번역한 문장:  pigé ? 
-----------------------------------
입력 문장: Hang on.
정답 문장:  Tiens bon ! 
번역기가 번역한 문장:  accroche-toi ! 
-----------------------------------
입력 문장: Here's $5.
정답 문장:  Voilà cinq dollars. 
번역기가 번역한 문장:  voici. 


비슷한 의미를 번역해주었다.

# 단어 수준의 번역기 만들기

앞서 만들어본 글자 수준의 번역기는 단위 단위로 많이 구현된다. 단어단위로 하면 단어사전의 크기도 커지고 학습속도도 더 느려진다. 그래서 데이터 상위 33,000개의 샘플(3천개 테스트 데이터)만을 사용해볼 것이다.

In [28]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [29]:
import tensorflow as tf

print(tf.__version__)

2.6.0


In [30]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines_w = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines_w))
lines_w.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
81247,It doesn't matter anymore.,Ça n'a plus d'importance.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
89980,Tell me why you think that.,Dis-moi pourquoi tu penses ça.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
96553,"I'm sorry, my father is out.","Je suis désolée, mon père est éliminé.",CC-BY 2.0 (France) Attribution: tatoeba.org #7...
169027,I've been reading that book all afternoon.,J'ai lu ce livre toute l'après-midi.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
97751,Tell me the meaning of life.,Dis-moi le sens de la vie.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [31]:
lines_w = lines_w[['eng', 'fra']][:33000] # 5만개 샘플 사용
lines_w.sample(5)

,eng,fra
21325,Tom was drowning.,Tom se noyait.
8615,There you are.,Vous y voici.
14008,I didn't listen.,Je n'ai pas écouté.
6931,Give it to me.,Donne-la-moi.
32999,We are watching TV.,Nous regardons la télévision.


## 1. 정제, 정규화, 전처리

### 1. 구두점을 단어와 분리한다.
토큰화할 때 띄어스기 단위로 한다면, 구두점이 문자와 같이 들어가 구분되길 바라질 않고 분리되어지길 바란다. 띄어쓰기를 해주자.

### 2. 소문자로 바꾼다.
컴퓨터가 보이에는 A와 a는 다르다.

### 3. 띄어쓰기 단위로 토큰화 한다.

케라스의 문자를 단어의 시퀀스로 만들어주는 메서드를 활용해보자.

In [32]:
lines_w['eng'][5000]

'I like fruit.'

In [33]:
print(tf.keras.preprocessing.text.text_to_word_sequence(lines_w['eng'][5000],
                                               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                               lower=True,
                                               split=" "))

['i', 'like', 'fruit']


구두점들은 잘 처리해서 소문자로 바꿔주었다. 띄어쓰기 단위로 토큰화도 되었다.

데이터 프레임의 단어사전의 데이터에 적용해보자

In [34]:
lines_w.iloc[5000][1]

"J'aime les fruits."

In [35]:
len(lines_w.iloc[5000])

2

In [36]:
result_en = []
for i in range(len(lines_w)):
    row = tf.keras.preprocessing.text.text_to_word_sequence(lines_w.iloc[i][0],
                                                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                       lower=True,
                                                       split=" ")
    result_en.append(row)
print(pd.Series(result_en))

0                            [go]
1                            [go]
2                            [go]
3                            [go]
4                            [hi]
                   ...           
32995    [we, all, cried, a, lot]
32996     [we, all, felt, hungry]
32997     [we, also, found, this]
32998     [we, are, busy, people]
32999     [we, are, watching, tv]
Length: 33000, dtype: object


In [37]:
result_fr = []
for i in range(len(lines_w)):
    row = tf.keras.preprocessing.text.text_to_word_sequence(lines_w.iloc[i][1],
                                                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                       lower=True,
                                                       split=" ")
    result_fr.append(row)
print(pd.Series(result_fr))

0                                           [va]
1                                       [marche]
2                                    [en, route]
3                                        [bouge]
4                                        [salut]
                          ...                   
32995    [nous, avons, toutes, beaucoup, pleuré]
32996                 [nous, avions, tous, faim]
32997         [nous, avons, aussi, trouvé, ceci]
32998         [nous, sommes, des, gens, occupés]
32999          [nous, regardons, la, télévision]
Length: 33000, dtype: object


In [38]:
input_data = pd.Series(result_en)
target_data = pd.Series(result_fr)
print(input_data.head())
print(target_data.head())

0    [go]
1    [go]
2    [go]
3    [go]
4    [hi]
dtype: object
0           [va]
1       [marche]
2    [en, route]
3        [bouge]
4        [salut]
dtype: object


In [39]:
type(input_data[0])

list

In [40]:
df_en = pd.DataFrame(input_data)
df_en.columns = ['eng']
df_fr = pd.DataFrame(target_data)
df_fr.columns = ['fra']
print(df_en.head())
print(df_fr.head())

    eng
0  [go]
1  [go]
2  [go]
3  [go]
4  [hi]
           fra
0         [va]
1     [marche]
2  [en, route]
3      [bouge]
4      [salut]


In [41]:
df_fr.shape

(33000, 1)

In [42]:
df_data= pd.DataFrame({'eng': result_en, 'fra': result_fr})
df_data.head()

,eng,fra
0,[go],[va]
1,[go],[marche]
2,[go],"[en, route]"
3,[go],[bouge]
4,[hi],[salut]


## 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주자.
글자 단위 번역기를 구현한 것처럼 디코더의 입력 시퀀스 맨 앞에는 시작을 의미하는 토큰 <sos>가 필요하다. 그리고 teaching forcing을 하는데 종료 토큰 <eos>가 필요하다.

In [43]:
sos_token = '<sos>'
eos_token = '<eos>'
result_fr = []
for i in range(len(lines_w)):
    row = tf.keras.preprocessing.text.text_to_word_sequence(lines_w.iloc[i][1],
                                                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                       lower=True,
                                                       split=" ")
    row.insert(0,sos_token)
    row.append(eos_token)
    result_fr.append(row)
print(pd.Series(result_fr))

0                                       [<sos>, va, <eos>]
1                                   [<sos>, marche, <eos>]
2                                [<sos>, en, route, <eos>]
3                                    [<sos>, bouge, <eos>]
4                                    [<sos>, salut, <eos>]
                               ...                        
32995    [<sos>, nous, avons, toutes, beaucoup, pleuré,...
32996             [<sos>, nous, avions, tous, faim, <eos>]
32997     [<sos>, nous, avons, aussi, trouvé, ceci, <eos>]
32998     [<sos>, nous, sommes, des, gens, occupés, <eos>]
32999      [<sos>, nous, regardons, la, télévision, <eos>]
Length: 33000, dtype: object


In [44]:
df_data= pd.DataFrame({'eng': result_en, 'fra': result_fr})
df_data.head()

,eng,fra
0,[go],"[<sos>, va, <eos>]"
1,[go],"[<sos>, marche, <eos>]"
2,[go],"[<sos>, en, route, <eos>]"
3,[go],"[<sos>, bouge, <eos>]"
4,[hi],"[<sos>, salut, <eos>]"


## 3. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보자.
영어와 프랑스어에 대한 토크나이저를 각각 생성하고, tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환한다.

In [45]:
df_data['fra'][0]

['<sos>', 'va', '<eos>']

In [46]:
fra_tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
fra_tokenizer.fit_on_texts(df_data['fra']) 

fra_tokenizer.word_index

{'<sos>': 1,
 '<eos>': 2,
 'je': 3,
 'tom': 4,
 'vous': 5,
 'est': 6,
 'pas': 7,
 'il': 8,
 'le': 9,
 'de': 10,
 'nous': 11,
 'ne': 12,
 'suis': 13,
 'tu': 14,
 "c'est": 15,
 'la': 16,
 'a': 17,
 "j'ai": 18,
 'à': 19,
 'un': 20,
 'ce': 21,
 'en': 22,
 'me': 23,
 'êtes': 24,
 'ça': 25,
 'que': 26,
 'les': 27,
 'une': 28,
 'moi': 29,
 'es': 30,
 'elle': 31,
 'sont': 32,
 'ils': 33,
 'sommes': 34,
 'fait': 35,
 'tout': 36,
 'elles': 37,
 'qui': 38,
 'des': 39,
 "n'est": 40,
 'mon': 41,
 'te': 42,
 'toi': 43,
 'y': 44,
 'bien': 45,
 'était': 46,
 'très': 47,
 'été': 48,
 'se': 49,
 'du': 50,
 'besoin': 51,
 "l'air": 52,
 'faire': 53,
 'ici': 54,
 'peux': 55,
 'personne': 56,
 'veux': 57,
 'votre': 58,
 'va': 59,
 'as': 60,
 'cela': 61,
 'ai': 62,
 'fais': 63,
 "j'aime": 64,
 "s'est": 65,
 'on': 66,
 'ton': 67,
 'faut': 68,
 'là': 69,
 'avez': 70,
 'avons': 71,
 'comment': 72,
 'tous': 73,
 'ont': 74,
 'ma': 75,
 'trop': 76,
 'monde': 77,
 "c'était": 78,
 'au': 79,
 'aller': 80,
 'pour': 81

In [47]:
eng_tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
eng_tokenizer.fit_on_texts(df_data['eng']) 

eng_tokenizer.word_index

{'i': 1,
 'you': 2,
 'tom': 3,
 'is': 4,
 'a': 5,
 'it': 6,
 "i'm": 7,
 'the': 8,
 'was': 9,
 'me': 10,
 "you're": 11,
 'this': 12,
 'he': 13,
 'are': 14,
 'to': 15,
 'that': 16,
 'we': 17,
 'do': 18,
 "it's": 19,
 'my': 20,
 'your': 21,
 'go': 22,
 'have': 23,
 'not': 24,
 "don't": 25,
 'they': 26,
 'be': 27,
 'no': 28,
 "we're": 29,
 'can': 30,
 'did': 31,
 'she': 32,
 'all': 33,
 'like': 34,
 'here': 35,
 'in': 36,
 'get': 37,
 "that's": 38,
 'up': 39,
 "i'll": 40,
 'very': 41,
 'on': 42,
 'what': 43,
 'need': 44,
 'how': 45,
 'want': 46,
 "they're": 47,
 'one': 48,
 'him': 49,
 'out': 50,
 'love': 51,
 'come': 52,
 'please': 53,
 "can't": 54,
 'us': 55,
 'just': 56,
 'know': 57,
 'now': 58,
 'stop': 59,
 'so': 60,
 'of': 61,
 "let's": 62,
 'got': 63,
 'too': 64,
 'help': 65,
 "he's": 66,
 'look': 67,
 'take': 68,
 'let': 69,
 'good': 70,
 'for': 71,
 'who': 72,
 'has': 73,
 'there': 74,
 'at': 75,
 'see': 76,
 'will': 77,
 'were': 78,
 'keep': 79,
 'had': 80,
 'am': 81,
 "tom's": 8

In [48]:
print(eng_tokenizer.word_counts)

OrderedDict([('go', 815), ('hi', 14), ('run', 71), ('who', 244), ('wow', 5), ('duck', 4), ('fire', 17), ('help', 275), ('hide', 10), ('jump', 16), ('stop', 290), ('wait', 119), ('begin', 13), ('on', 426), ('hello', 16), ('i', 6331), ('see', 232), ('try', 187), ('won', 57), ('oh', 12), ('no', 627), ('relax', 23), ('smile', 17), ('sorry', 46), ('attack', 14), ('buy', 53), ('it', 2435), ('cheers', 4), ('eat', 144), ('get', 462), ('up', 445), ('now', 291), ('got', 279), ('hop', 2), ('in', 490), ('hug', 16), ('me', 1245), ('fell', 42), ('fled', 1), ('knit', 4), ('know', 296), ('left', 100), ('lied', 27), ('lost', 150), ('paid', 32), ('pass', 10), ('quit', 53), ("i'm", 1835), ('19', 2), ('ok', 77), ('listen', 41), ('way', 83), ('really', 97), ('thanks', 44), ('we', 1029), ('ask', 65), ('tom', 3441), ('him', 357), ('awesome', 31), ('be', 651), ('calm', 46), ('cool', 47), ('fair', 25), ('kind', 38), ('nice', 121), ('beat', 33), ('burn', 8), ('bury', 5), ('call', 99), ('us', 300), ('come', 321)

In [49]:
input_text = eng_tokenizer.texts_to_sequences(df_data['eng'])
print(input_text)

[[22], [22], [22], [22], [778], [778], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [203], [72], [1553], [1553], [1553], [1770], [1770], [1770], [676], [65], [989], [989], [711], [711], [59], [59], [59], [128], [128], [128], [128], [128], [128], [128], [830], [830], [22, 42], [22, 42], [22, 42], [712], [712], [1, 76], [1, 76], [1, 87], [1, 240], [1, 240], [1, 240], [872, 28], [545], [545], [545], [545], [545], [545], [545], [545], [545], [545], [545], [545], [677], [677], [677], [289], [779], [779], [779], [779], [253, 6], [253, 6], [253, 6], [253, 6], [1771], [1771], [1771], [1771], [111, 6], [111, 6], [37, 39], [37, 39], [37, 39], [22, 58], [22, 58], [22, 58], [63, 6], [63, 6], [63, 6], [63, 6], [63, 6], [63, 6], [2527, 36], [2527, 36], [713, 10], [713, 10], [1, 311], [1, 311], [1, 3355], [1, 1772], [1, 57], [1, 144], [1, 144], [1, 478], [1, 107], [1, 414], [1, 414], [1, 414], [1, 990], [1, 254], [1, 254], [7, 2528], [7, 183

In [50]:
target_text = fra_tokenizer.texts_to_sequences(df_data['fra'])
print(target_text)

[[1, 59, 2], [1, 351, 2], [1, 22, 499, 2], [1, 730, 2], [1, 776, 2], [1, 776, 2], [1, 3652, 2], [1, 3653, 2], [1, 171, 243, 1011, 19, 243, 2810, 2], [1, 1463, 2], [1, 2293, 2], [1, 701, 2], [1, 2811, 2], [1, 3654, 2], [1, 3652, 2], [1, 3653, 2], [1, 171, 243, 1011, 19, 243, 2810, 2], [1, 1463, 2], [1, 2293, 2], [1, 701, 2], [1, 2811, 2], [1, 3654, 2], [1, 38, 2], [1, 25, 5477, 2], [1, 5478, 2], [1, 5479, 2], [1, 19, 5480, 2], [1, 1309, 1464, 2], [1, 3655, 1012, 2], [1, 79, 484, 2], [1, 19, 2812, 2], [1, 1663, 43, 2], [1, 5481, 5, 2], [1, 1465, 2], [1, 1465, 2], [1, 25, 3656, 2], [1, 5482, 2], [1, 133, 43, 2], [1, 400, 2], [1, 438, 2], [1, 438, 2], [1, 400, 2], [1, 438, 2], [1, 400, 2], [1, 438, 2], [1, 1102, 2], [1, 702, 2], [1, 1664, 2], [1, 357, 2], [1, 2813, 2], [1, 703, 2], [1, 776, 2], [1, 3, 665, 2], [1, 3657, 2], [1, 2294, 2], [1, 18, 295, 2], [1, 3, 89, 934, 2], [1, 178, 295, 2], [1, 1310, 186, 2], [1, 216, 43, 2], [1, 1103, 877, 2], [1, 1908, 439, 2], [1, 1909, 3658, 2], [1, 7

In [75]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [76]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 59], [1, 351], [1, 22, 499]]
[[59, 2], [351, 2], [22, 499, 2]]


In [77]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 6)
프랑스어 입력데이터의 크기(shape) : (33000, 14)
프랑스어 출력데이터의 크기(shape) : (33000, 14)


In [78]:
print(encoder_input[0])

[22  0  0  0  0  0]


In [65]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 6, 4799)
프랑스어 입력데이터의 크기(shape) : (33000, 14, 10009)
프랑스어 출력데이터의 크기(shape) : (33000, 14, 10009)


### 벨리데이션 데이터셋 분리하기

In [79]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [87]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [88]:
print("encoder_input_train.shape :", encoder_input_train.shape)
print("decoder_input_train.shape :", decoder_input_train.shape)
print("decoder_target_train.shape :", decoder_target_train.shape)
print()
print("encoder_input_test.shape :", encoder_input_test.shape)
print("decoder_input_test.shape :", decoder_input_test.shape)
print("decoder_target_test.shape :", decoder_target_test.shape)

encoder_input_train.shape : (29700, 6)
decoder_input_train.shape : (29700, 14)
decoder_target_train.shape : (29700, 14)

encoder_input_test.shape : (3300, 6)
decoder_input_test.shape : (3300, 14)
decoder_target_test.shape : (3300, 14)


## 4. 임베딩 층(Embedding layer) 사용하기

In [89]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4799
프랑스어 단어장의 크기 : 10009


In [90]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 6
프랑스어 시퀀스의 최대 길이 14


In [91]:
print('전체 샘플의 수 :',len(lines_w))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4799
프랑스어 단어장의 크기 : 10009
영어 시퀀스의 최대 길이 6
프랑스어 시퀀스의 최대 길이 14


### 인코더 임베딩

In [92]:
embedding_dim = 64
hidden_units = 64

In [93]:
from tensorflow.keras.layers import Input, Embedding, Masking


encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
encoder_lstm = tf.keras.layers.LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

케라스의 임베딩 레이어의 파라미터를 살펴보자.

~~~python
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)
~~~

### 디코더 임베딩

In [94]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(fra_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = tf.keras.layers.LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_emb,initial_state=encoder_states)

decoder_dense = tf.keras.layers.Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## 5. 모델 구현하기

In [95]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [97]:
trans_hist = model.fit(
    x=[encoder_input_train, decoder_input_train], 
    y=decoder_target_train,
    validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
    batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 5s 23ms/step - loss: 1.7449 - acc: 0.7450 - val_loss: 2.0419 - val_acc: 0.7062
Epoch 2/50
233/233 [==============================] - 5s 23ms/step - loss: 1.6339 - acc: 0.7541 - val_loss: 1.9483 - val_acc: 0.7231
Epoch 3/50
233/233 [==============================] - 5s 22ms/step - loss: 1.5326 - acc: 0.7645 - val_loss: 1.8595 - val_acc: 0.7316
Epoch 4/50
233/233 [==============================] - 5s 23ms/step - loss: 1.4305 - acc: 0.7803 - val_loss: 1.7837 - val_acc: 0.7391
Epoch 5/50
233/233 [==============================] - 5s 22ms/step - loss: 1.3481 - acc: 0.7916 - val_loss: 1.7152 - val_acc: 0.7487
Epoch 6/50
233/233 [==============================] - 5s 23ms/step - loss: 1.2785 - acc: 0.8004 - val_loss: 1.6668 - val_acc: 0.7584
Epoch 7/50
233/233 [==============================] - 5s 23ms/step - loss: 1.2146 - acc: 0.8078 - val_loss: 1.6199 - val_acc: 0.7649
Epoch 8/50
233/233 [==============================] - 5s 22ms/step - 

In [98]:
import tensorflow as tf
import matplotlib.pyplot as plt
def hist_plot(history:tf.keras.callbacks.History()):

    tacc = history.history["acc"]
    vacc = history.history["val_acc"]

    tloss=history.history["loss"]
    vloss=history.history["val_loss"]

    epochs_range = range(50)

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, tacc, label="Training Accuracy")
    plt.plot(epochs_range, vacc, label="Validation Accuracy")
    plt.legend(loc="lower right")
    plt.title("Training and Validation Accuracy")

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, tloss, label="Training Loss")
    plt.plot(epochs_range, vloss, label="Validation Loss")
    plt.legend(loc="upper right")
    plt.title("Training and Validation Loss")
    
    plt.show()

## 6. 모델 평가하기

In [104]:
src_to_index = eng_tokenizer.word_index
index_to_src = eng_tokenizer.index_word
tar_to_index = fra_tokenizer.word_index
index_to_tar = fra_tokenizer.index_word

In [105]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [106]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [107]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

## 훈련 데이터에서 임의의 샘플을 추출해서 평가

In [108]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : go 
정답문장 : bouge 
번역문장 : va 
--------------------------------------------------
입력문장 : hello 
정답문장 : bonjour 
번역문장 : bonjour 
--------------------------------------------------
입력문장 : got it 
정답문장 : t'as capté  
번역문장 : ça va 
--------------------------------------------------
입력문장 : hang on 
정답문장 : tiens bon 
번역문장 : attendez 
--------------------------------------------------
입력문장 : here's 5 
정답문장 : voilà cinq dollars 
번역문장 : voilà cinq dollars 
--------------------------------------------------


입력문장 : go 
정답문장 : bouge (move)
번역문장 : va (vain) 조금 어색함
--------------------------------------------------
입력문장 : hello 
정답문장 : bonjour 
번역문장 : bonjour 정답
--------------------------------------------------
입력문장 : got it 
정답문장 : t'as capté (you caught)
번역문장 : ça va (all right) 비슷하다
--------------------------------------------------
입력문장 : hang on 
정답문장 : tiens bon 
번역문장 : attendez (Hold on) 비슷하다.
--------------------------------------------------
입력문장 : here's 5 
정답문장 : voilà cinq dollars 
번역문장 : voilà cinq dollars (That's five dollars) 정답과 비슷하다.
--------------------------------------------------

## 테스트 데이터를 가지고 평가

In [109]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : i fell off my bike 
정답문장 : je suis tombé de vélo 
번역문장 : je suis tombé pas une faveur 
--------------------------------------------------
입력문장 : i had loads of fun 
정답문장 : je me suis bien marrée 
번역문장 : je me suis amusé 
--------------------------------------------------
입력문장 : i have blonde hair 
정답문장 : j'ai les cheveux blonds 
번역문장 : j'ai les cheveux châtains 
--------------------------------------------------
입력문장 : i lost your number 
정답문장 : j'ai perdu ton numéro 
번역문장 : je me sens très bien 
--------------------------------------------------
입력문장 : i'm so embarrassed 
정답문장 : je suis tellement embarrassée 
번역문장 : je suis à proximité 
--------------------------------------------------


입력문장 : i fell off my bike 
정답문장 : je suis tombé de vélo 
번역문장 : je suis tombé pas une faveur (I fell not a favor) 조금 어색하다.
--------------------------------------------------
입력문장 : i had loads of fun 
정답문장 : je me suis bien marrée 
번역문장 : je me suis amusé (I had fun) 정답과 비슷하다.
--------------------------------------------------
입력문장 : i have blonde hair 
정답문장 : j'ai les cheveux blonds 
번역문장 : j'ai les cheveux châtains (I have brown hair) 정답과 유사하다.
--------------------------------------------------
입력문장 : i lost your number 
정답문장 : j'ai perdu ton numéro 
번역문장 : je me sens très bien (
I feel very good) 이건 제대로 되지 않았다.
--------------------------------------------------
입력문장 : i'm so embarrassed 
정답문장 : je suis tellement embarrassée 
번역문장 : je suis à proximité (I'm nearby) 이것도 좀 어색하다.
--------------------------------------------------


임의의 테스트 데이터 5개에서 2개만 비슷했다. 아쉬운 결과다.

# 회고

## 발생한 문제
- 패딩을 넣어줄 때, 'post'로 지정해봤는데, 테스트를 할 때, Go 까지 출력이 되면서 키에러0 이 떴다.
    - 번역기가 번역한 문장도 같은 단어가 수 번 연달하서 입력되어 출력되었다.
- tf.keras.preprocee의 전처리 메서드를 활용하려고 했는데, DataFrame으로 적용할 수 없었다.
    - for loop를 활용해서 데이터를 호출해내서 처리하고 다시 데이터프레임으로 넣었다. 여기서도 병합하는데 어려움이 있었다.


3만개의 적은 데이터로 학습해서 나름 훌륭한 결과라고 볼 수 있을 거같다. 그래도 훈련데이터에서는 높은 정확도를 보여줬는데 아직 데이터가 부족해서 인지 테스트 데이터에서는 훌륭한 결과라고 보기 어려웠다. 그래도 앞으로 더 최신 기술들을 배워가고 데이터셋의 크기도 키워본다면 좋은 결과가 있을거 같다.

단어를 임베딩하고 인코더RNN과 디코더RNN를 결합하여 실습해봤다.
이론적으로 좀 이해하게 되었지만, 아직은 이해하는데 한계가 있었다. 좀 더 다양하게 실습해보면서 이론도 정립해나가야 겠다.